In [6]:
import pandas as pd

train = pd.read_csv("data/train.csv")

train["Hyp"] = 0
train.loc[train.Sex == 'female', "Hyp"] = 1

train["Result"] = 0
train.loc[train.Survived == train["Hyp"], "Result"] = 1

print(train["Result"].value_counts())

1    701
0    190
Name: Result, dtype: int64


In [12]:
#使用logistic回归进行预测，仅在训练集上对比
#没有特别考虑性别权重
import pandas as pd
from sklearn import linear_model, preprocessing

def clean_data(data):
    data["Fare"] = data["Fare"].fillna(data["Fare"].dropna().median()) #用非空中值填补空的票价
    data["Age"] = data["Age"].fillna(data["Age"].dropna().median())
    
    data.loc[data["Sex"] == "male", "Sex"] = 0 #数值化性别
    data.loc[data["Sex"] == "female", "Sex"] = 1
    
    data["Embarked"] = data["Embarked"].fillna("S") #用S填补空的登船地点
    data.loc[data["Embarked"] == "S", "Embarked"] = 0
    data.loc[data["Embarked"] == "C", "Embarked"] = 1
    data.loc[data["Embarked"] == "Q", "Embarked"] = 2

train = pd.read_csv("data/train.csv")
clean_data(train)
feature_names = ["Pclass", "Age", "Sex", "SibSp", "Parch"]

target = train["Survived"].values
features = train[feature_names].values
classifier = linear_model.LogisticRegression()
classifier_ = classifier.fit(features, target)
print(classifier_.score(features, target))

'''Generate polynomial and interaction features.
Generate a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to the specified degree. 
For example, if an input sample is two dimensional and of the form [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2].
'''
poly = preprocessing.PolynomialFeatures(degree=2) #创造更多特征，补充2次型
poly_features = poly.fit_transform(features)

classifier_ = classifier.fit(poly_features, target)
print(classifier_.score(poly_features, target))

0.793490460157
0.830527497194


In [27]:
#使用决策树预测，仅在训练集上对比
#性别没有特殊权重
import pandas as pd
from sklearn import tree, model_selection
import numpy as np

def clean_data(data):
    data["Fare"] = data["Fare"].fillna(data["Fare"].dropna().median()) #用非空中值填补空的票价
    data["Age"] = data["Age"].fillna(data["Age"].dropna().median())
    
    data.loc[data["Sex"] == "male", "Sex"] = 0 #数值化性别
    data.loc[data["Sex"] == "female", "Sex"] = 1
    
    data["Embarked"] = data["Embarked"].fillna("S") #用S填补空的登船地点
    data.loc[data["Embarked"] == "S", "Embarked"] = 0
    data.loc[data["Embarked"] == "C", "Embarked"] = 1
    data.loc[data["Embarked"] == "Q", "Embarked"] = 2

def write_prediction(prediction, name):
    PassengerId = np.array(test["PassengerId"]).astype(int)
    solution = pd.DataFrame(prediction, PassengerId, columns = ["Survived"])
    solution.to_csv(name, index_label = ["PassengerId"])

train = pd.read_csv("data/train.csv")
clean_data(train)
feature_names = ["Pclass", "Age", "Sex", "SibSp", "Parch"]

target = train["Survived"].values
features = train[feature_names].values

decision_tree = tree.DecisionTreeClassifier(random_state = 1)
decision_tree_ = decision_tree.fit(features,target)

print("决策数得分：",decision_tree_.score(features,target))
#验证某个模型在某个训练集上的稳定性，输出k个预测精度。
scores = model_selection.cross_val_score(decision_tree, features, target, scoring='accuracy',cv=50)
print("scores:",scores)
print("avg scores:",scores.mean())

print("==========================================")
#防止过拟合？
generalized_tree = tree.DecisionTreeClassifier(
    random_state = 1,
    max_depth = 7,
    min_samples_split = 2)
generalized_tree_ = generalized_tree.fit(features,target)
print("决策数得分：",generalized_tree_.score(features,target))
scores = model_selection.cross_val_score(generalized_tree, features, target, scoring='accuracy',cv=50)
print("scores:",scores)
print("avg scores:",scores.mean())

tree.export_graphviz(generalized_tree_,feature_names=feature_names, out_file="tree.dot")


#输出预测结果
test = pd.read_csv("data/test.csv")
clean_data(test)
test_features_two = test[feature_names].values
prediction_two = generalized_tree_.predict(test_features_two)
write_prediction(prediction_two, "decision_tree_two.csv")

决策数得分： 0.916947250281
scores: [ 0.77777778  0.72222222  0.77777778  0.88888889  0.77777778  0.77777778
  0.61111111  0.77777778  0.77777778  0.88888889  0.77777778  0.72222222
  0.77777778  0.77777778  0.77777778  0.77777778  0.94444444  0.83333333
  0.83333333  0.88888889  0.94444444  0.72222222  0.83333333  0.77777778
  0.72222222  0.83333333  0.83333333  0.88888889  0.55555556  0.83333333
  0.77777778  0.66666667  0.88888889  0.94444444  0.83333333  0.72222222
  0.66666667  0.72222222  0.83333333  0.88888889  0.88888889  0.83333333
  0.76470588  0.76470588  0.94117647  0.76470588  0.82352941  0.82352941
  0.82352941  0.875     ]
avg scores: 0.801617647059
决策数得分： 0.852974186308
scores: [ 0.88888889  0.72222222  0.77777778  0.94444444  0.72222222  0.83333333
  0.55555556  0.77777778  0.77777778  0.94444444  0.83333333  0.77777778
  0.83333333  0.77777778  0.66666667  0.66666667  0.94444444  0.77777778
  0.88888889  0.94444444  0.94444444  0.72222222  0.77777778  0.77777778
  0.7777777

In [32]:
import numpy as np
import pandas as pd
from sklearn import ensemble, model_selection, grid_search

def clean_data(data):
    data["Fare"] = data["Fare"].fillna(data["Fare"].dropna().median()) #用非空中值填补空的票价
    data["Age"] = data["Age"].fillna(data["Age"].dropna().median())
    
    data.loc[data["Sex"] == "male", "Sex"] = 0 #数值化性别
    data.loc[data["Sex"] == "female", "Sex"] = 1
    
    data["Embarked"] = data["Embarked"].fillna("S") #用S填补空的登船地点
    data.loc[data["Embarked"] == "S", "Embarked"] = 0
    data.loc[data["Embarked"] == "C", "Embarked"] = 1
    data.loc[data["Embarked"] == "Q", "Embarked"] = 2

def write_prediction(prediction, name):
    PassengerId = np.array(test["PassengerId"]).astype(int)
    solution = pd.DataFrame(prediction, PassengerId, columns = ["Survived"])
    solution.to_csv(name, index_label = ["PassengerId"])



train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

print("\nCleaning up some data")

clean_data(train)
clean_data(test)

print("\nExtracting target and features")

print(train.shape)
target = train["Survived"].values
features = train[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values

print("\nUse gradient boosting classifier")

# grid_search = grid_search.GridSearchCV(
#     estimator = ensemble.GradientBoostingClassifier(
#         learning_rate=0.001,
#         min_samples_split=40,
#         min_samples_leaf=1,
#         max_features=2,
#         max_depth=12,
#         n_estimators=70,
#         subsample=0.75,
#         random_state=10), 
#     param_grid = {'n_estimators':[140, 280, 560, 1120, 4480]},
#     scoring='roc_auc',
#     n_jobs=4,
#     iid=False,
#     cv=10)

# grid_search.fit(features, target)
# print(grid_search.grid_scores_, grid_search.best_params_, grid_search.best_score_)

gbm = ensemble.GradientBoostingClassifier(
    learning_rate = 0.005,
    min_samples_split=40,
    min_samples_leaf=1,
    max_features=2,
    max_depth=12,
    n_estimators=1500,
    subsample=0.75,
    random_state=1)
gbm = gbm.fit(features, target)

print(gbm.feature_importances_)
print(gbm.score(features, target))

# scores = model_selection.cross_val_score(gbm, features, target, scoring='accuracy', cv=20)
# print scores
# print scores.mean()

test_features = test[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
prediction_gbm = gbm.predict(test_features)
write_prediction(prediction_gbm, "gbm.csv")



Cleaning up some data

Extracting target and features
(891, 12)

Use gradient boosting classifier
[mean: 0.87306, std: 0.04514, params: {'n_estimators': 140}, mean: 0.87388, std: 0.04394, params: {'n_estimators': 280}, mean: 0.87483, std: 0.04443, params: {'n_estimators': 560}, mean: 0.87585, std: 0.04575, params: {'n_estimators': 1120}, mean: 0.87516, std: 0.05115, params: {'n_estimators': 4480}] {'n_estimators': 1120} 0.875848456554339
[ 0.05732547  0.32300303  0.11482147  0.36183847  0.05657822  0.04783569
  0.03859766]
0.94051627385
